# Spreadsheet comparison and grading

This notebook holds the exploratory code for comparing submissions against the key. It will also do grading.

## Key preparation

In [3]:
from openpyxl import load_workbook

# Loading the key workbook.
# The compared data will come from Sheet3, cell B6.

key_data = load_workbook("../sample_excel_files/type_1_key.xlsx", data_only=True)
key_formulas = load_workbook("../sample_excel_files/type_1_key.xlsx", data_only=False)

print(f"All sheetnames: {key_data.sheetnames}")

# This way, we assume all necessary sheets will have the _CheckOrder pair.
grading_sheetnames = [name for name in key_data.sheetnames if name.find("_CheckOrder") == -1]
print(f"Grading sheetnames: {grading_sheetnames}")


All sheetnames: ['Sheet3', 'Sheet3_CheckOrder']
Grading sheetnames: ['Sheet3']


## Key formula and grade parsing

In [35]:
# Parsing methods


def get_grading_sheetnames(key_wb):
    """
    Returns grading sheetnames from the passed key workbook.
    """
    return [name for name in key_wb.sheetnames if name.find("_CheckOrder") == -1]

def get_sheet_grading_sequence(key_wb):
    """
    Returns a dictionary of sheetnames as key. Each key will have an ordered list of tuples,
    which contains the cell name in that sheet and the grading rubric. Both are in strings.
    """
    grading_dict = {}
    sheetnames = get_grading_sheetnames(key_wb)
    
    for name in sheetnames:
        order_sheet = key_wb[name + "_CheckOrder"]
        key_sheet = key_wb[name]
        
        # Assumptions of the order sheet 
        # 1. The scoring column is always on B. (min_col=2, max_col=2)
        # 2. The scoring column always has a header (min_row=2)
        # 3. The scoring column is always in order        
        for column in order_sheet.iter_cols(min_col=2, max_col=2, min_row=2):
            # Assuming this for-loop will only be executed for B column
            # And the reference cells always have comment text for rubric.
            grading_dict[name] = [(cell.value, key_sheet[cell.value].comment.text) for cell in column]

    return grading_dict


In [36]:
# Test the methods above

key_grading_sequence = get_sheet_grading_sequence(key_data)

print("Key grading sequence for each sheet:")

for sheetname in key_grading_sequence:
    print(sheetname + ":")
    print(key_grading_sequence[sheetname])

Key grading sequence for each sheet:
Sheet3:
[('B6', 'Rubric:\n\t10P-C\n'), ('B7', 'Rubric:\n\t5P-C\n')]
